In [1]:
import pandas as pd

In [2]:
rom_korr_map = pd.read_csv("rom_korr_full_website_coords.csv")

In [7]:
#pip install folium

Note: you may need to restart the kernel to use updated packages.


In [7]:
import folium
import pandas as pd

def generate_map(df):
    """
    Erzeugt eine interaktive Folium-Karte aus einem DataFrame mit Spalten:
      - Date
      - Sender
      - Recipient
      - Place of Dispatch, Dispatch_Lat, Dispatch_Lon
      - Place of Destination, Destination_Lat, Destination_Lon
      - Distance_km (optional)
      - link (optional)

    Gibt ein folium.Map-Objekt zurück.
    """

    # 1) Einen Startpunkt für die Karte berechnen (Mittelwert aller Koordinaten).
    valid_lat = df['Dispatch_Lat'].dropna().tolist() + df['Destination_Lat'].dropna().tolist()
    valid_lon = df['Dispatch_Lon'].dropna().tolist() + df['Destination_Lon'].dropna().tolist()

    if len(valid_lat) > 0 and len(valid_lon) > 0:
        avg_lat = sum(valid_lat) / len(valid_lat)
        avg_lon = sum(valid_lon) / len(valid_lon)
    else:
        # Fallback, wenn keine Koordinaten vorhanden sind
        avg_lat, avg_lon = 50.0, 10.0  # Mitteleuropa

    # 2) Neue Folium-Karte anlegen
    my_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    # 3) Über alle Zeilen iterieren
    for idx, row in df.iterrows():
        disp_lat = row.get("Dispatch_Lat")
        disp_lon = row.get("Dispatch_Lon")
        dest_lat = row.get("Destination_Lat")
        dest_lon = row.get("Destination_Lon")

        # Falls Koordinaten fehlen, diese Zeile überspringen
        if pd.isna(disp_lat) or pd.isna(disp_lon) or pd.isna(dest_lat) or pd.isna(dest_lon):
            continue

        # Popup-Infos für Absendeort
        popup_info_dispatch = f"""
        <b>{row.get('Sender', 'Unknown')}</b><br>
        <i>Absendeort:</i> {row.get('Place of Dispatch', 'Unknown')}<br>
        <i>Datum:</i> {row.get('Date', 'Unknown')}<br>
        <i>Link:</i> <a href="{row.get('link', '#')}" target="_blank">Original</a>
        """

        # Popup-Infos für Empfangsort
        popup_info_destination = f"""
        <b>{row.get('Recipient', 'Unknown')}</b><br>
        <i>Empfangsort:</i> {row.get('Place of Destination', 'Unknown')}<br>
        <i>Datum:</i> {row.get('Date', 'Unknown')}<br>
        <i>Link:</i> <a href="{row.get('link', '#')}" target="_blank">Original</a>
        """

        # Distance (optional)
        distance = row.get("Distance_km")
        if pd.notnull(distance):
            popup_info_dispatch += f"<br><i>Distanz:</i> ~{distance:.1f} km"
            popup_info_destination += f"<br><i>Distanz:</i> ~{distance:.1f} km"

        # 4) Roten Marker (Icon) für den Absendeort
        folium.Marker(
            location=[disp_lat, disp_lon],
            popup=folium.Popup(popup_info_dispatch, max_width=300),
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(my_map)

        # 5) Blauen Marker für den Empfangsort
        folium.Marker(
            location=[dest_lat, dest_lon],
            popup=folium.Popup(popup_info_destination, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(my_map)

        # 6) Linie zwischen den beiden Punkten
        folium.PolyLine(
            locations=[(disp_lat, disp_lon), (dest_lat, dest_lon)],
            color='green',
            weight=2,
            opacity=0.6
        ).add_to(my_map)

    return my_map


if __name__ == "__main__":

    # Hier angenommen, du hast deinen DataFrame schon parat:
    # rom_korr_full_website = pd.read_csv("deine_datei.csv")
    # Oder aus deinem aktuellen Notebook etc.

    # Beispiellos direkt die Variable an die Funktion übergeben:
    my_map = generate_map(rom_korr_map)

    # Karte als HTML speichern
    my_map.save("romantik_map.html")
    print("Karte wurde gespeichert als romantik_map.html!")


Karte wurde gespeichert als romantik_map.html!


In [8]:
import folium
import pandas as pd

def generate_map(df):
    """
    Erzeugt eine interaktive Folium-Karte aus einem DataFrame df,
    der folgende Spalten enthält:
      - Date
      - Sender
      - Recipient
      - Place of Dispatch, Dispatch_Lat, Dispatch_Lon
      - Place of Destination, Destination_Lat, Destination_Lon
      - Distance_km (optional)
      - link (optional)

    Die Popups zeigen:
      - Überschrift: "Absender an Empfänger"
      - Absende- und Empfangsort
      - Datum
      - Distanz (falls vorhanden)
      - Link zur Originalseite
    Zusätzlich wird ein unsichtbarer Marker in die Mitte jeder Linie gesetzt,
    damit man bei Klick auf diesen Marker ebenfalls ein Popup sieht.
    """

    # 1) Kartenmittelpunkt bestimmen (Durchschnitt aller Koordinaten)
    valid_lat = df['Dispatch_Lat'].dropna().tolist() + df['Destination_Lat'].dropna().tolist()
    valid_lon = df['Dispatch_Lon'].dropna().tolist() + df['Destination_Lon'].dropna().tolist()

    if len(valid_lat) > 0 and len(valid_lon) > 0:
        avg_lat = sum(valid_lat) / len(valid_lat)
        avg_lon = sum(valid_lon) / len(valid_lon)
    else:
        # Fallback, wenn keine Koordinaten vorliegen
        avg_lat, avg_lon = 50.0, 10.0

    # 2) Folium-Karte erzeugen
    my_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    # 3) Über alle Zeilen iterieren
    for idx, row in df.iterrows():
        disp_lat = row.get("Dispatch_Lat")
        disp_lon = row.get("Dispatch_Lon")
        dest_lat = row.get("Destination_Lat")
        dest_lon = row.get("Destination_Lon")

        # Wenn Koordinaten fehlen, Überspringen
        if pd.isna(disp_lat) or pd.isna(disp_lon) or pd.isna(dest_lat) or pd.isna(dest_lon):
            continue

        # Aus dem DataFrame gelesene Felder
        date_str = row.get('Date', 'Unknown')
        sender = row.get('Sender', 'Unknown')
        recipient = row.get('Recipient', 'Unknown')
        place_disp = row.get('Place of Dispatch', 'Unknown')
        place_dest = row.get('Place of Destination', 'Unknown')
        link_str = row.get('link', '#')

        # Distanz (optional)
        distance = row.get("Distance_km")
        dist_info = ""
        if pd.notnull(distance):
            dist_info = f"Distanz: ~{distance:.1f} km<br/>"

        # Überschrift im Popup: "Absender an Empfänger"
        heading = f"{sender} <b>an</b> {recipient}"

        # --- POPUP-Texte für Absende-/Empfangsort ---
        popup_info_dispatch = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        popup_info_destination = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        # --- POPUP für die Linie (unsichtbarer Marker in der Mitte) ---
        line_popup_html = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        # 4) Marker für den Absendeort (Rot)
        folium.Marker(
            location=[disp_lat, disp_lon],
            popup=folium.Popup(popup_info_dispatch, max_width=300),
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(my_map)

        # 5) Marker für den Empfangsort (Blau)
        folium.Marker(
            location=[dest_lat, dest_lon],
            popup=folium.Popup(popup_info_destination, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(my_map)

        # 6) Linie zwischen den beiden Orten
        folium.PolyLine(
            locations=[(disp_lat, disp_lon), (dest_lat, dest_lon)],
            color='green',
            weight=2,
            opacity=0.6,
            tooltip="Klicken für mehr Info"  # Wird beim Hover über die Linie angezeigt
        ).add_to(my_map)

        # Unsichtbarer Marker in der Mitte der Linie für ein weiteres Popup
        mid_lat = (disp_lat + dest_lat) / 2
        mid_lon = (disp_lon + dest_lon) / 2

        line_popup = folium.Popup(line_popup_html, max_width=350)
        folium.Marker(
            location=[mid_lat, mid_lon],
            popup=line_popup,
            icon=folium.DivIcon(icon_size=(0, 0))  # unsichtbarer Marker
        ).add_to(my_map)

    return my_map


if __name__ == "__main__":
    # Hier nimmst du deinen DataFrame "rom_korr_map".
    # Beispiel:
    # rom_korr_map = pd.read_csv("rom_korr_map.csv")
    # oder
    # rom_korr_map = rom_korr_map  # Falls er in deinem Notebook definiert ist

    # Die generate_map-Funktion mit dem DataFrame aufrufen:
    my_map = generate_map(rom_korr_map)

    # Karte speichern
    my_map.save("romantik_map_2.html")
    print("Karte erfolgreich unter romantik_map_2.html gespeichert!")


Karte erfolgreich unter romantik_map_2.html gespeichert!


In [9]:
import folium
import pandas as pd

def on_each_feature(feature, layer):
    """
    Diese Callback-Funktion bindet das Popup an die gesamte Linie
    (das 'layer'), damit ein Klick auf die Linie das Popup öffnet.
    """
    popup_text = feature["properties"]["popup"]
    layer.bindPopup(popup_text)


def add_line_geojson(map_obj, lat1, lon1, lat2, lon2, popup_html):
    """
    Erzeugt ein GeoJSON-LineString-Feature, das klickbar ist
    und ein Popup anzeigt. Achtung: In GeoJSON ist die Reihenfolge [LON, LAT]!
    """
    geojson_feature = {
        "type": "Feature",
        "properties": {
            "popup": popup_html  # Hier packen wir unseren Popup-Text rein
        },
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [lon1, lat1],
                [lon2, lat2]
            ]
        }
    }

    folium.GeoJson(
        data=geojson_feature,
        style_function=lambda x: {
            "color": "green",
            "weight": 2,
            "opacity": 0.6
        },
        on_each_feature=on_each_feature,  # Popup an die Linie binden
        tooltip="Klicken für mehr Info"   # Text beim Hover über die Linie
    ).add_to(map_obj)


def generate_map(df):
    """
    Erzeugt eine interaktive Folium-Karte aus einem DataFrame df,
    der folgende Spalten enthält:
      - Date
      - Sender
      - Recipient
      - Place of Dispatch, Dispatch_Lat, Dispatch_Lon
      - Place of Destination, Destination_Lat, Destination_Lon
      - Distance_km (optional)
      - link (optional)

    Die Popups zeigen:
      - Überschrift: "Absender an Empfänger"
      - Absende- und Empfangsort
      - Datum
      - Distanz (falls vorhanden)
      - Link zur Originalseite

    Anders als bisher verwenden wir für die Linie ein GeoJSON-LineString.
    So ist die komplette Linie klickbar und zeigt einen Popup an.
    """

    # 1) Kartenmittelpunkt bestimmen (Durchschnitt aller Koordinaten)
    valid_lat = df['Dispatch_Lat'].dropna().tolist() + df['Destination_Lat'].dropna().tolist()
    valid_lon = df['Dispatch_Lon'].dropna().tolist() + df['Destination_Lon'].dropna().tolist()

    if len(valid_lat) > 0 and len(valid_lon) > 0:
        avg_lat = sum(valid_lat) / len(valid_lat)
        avg_lon = sum(valid_lon) / len(valid_lon)
    else:
        # Fallback, wenn keine Koordinaten vorliegen
        avg_lat, avg_lon = 50.0, 10.0

    # 2) Folium-Karte erzeugen
    my_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    # 3) Über alle Zeilen iterieren
    for idx, row in df.iterrows():
        disp_lat = row.get("Dispatch_Lat")
        disp_lon = row.get("Dispatch_Lon")
        dest_lat = row.get("Destination_Lat")
        dest_lon = row.get("Destination_Lon")

        # Wenn Koordinaten fehlen, Überspringen
        if pd.isna(disp_lat) or pd.isna(disp_lon) or pd.isna(dest_lat) or pd.isna(dest_lon):
            continue

        # Aus dem DataFrame gelesene Felder
        date_str = row.get('Date', 'Unknown')
        sender = row.get('Sender', 'Unknown')
        recipient = row.get('Recipient', 'Unknown')
        place_disp = row.get('Place of Dispatch', 'Unknown')
        place_dest = row.get('Place of Destination', 'Unknown')
        link_str = row.get('link', '#')

        # Distanz (optional)
        distance = row.get("Distance_km")
        dist_info = ""
        if pd.notnull(distance):
            dist_info = f"Distanz: ~{distance:.1f} km<br/>"

        # Überschrift im Popup: "Absender an Empfänger"
        heading = f"{sender} <b>an</b> {recipient}"

        # --- POPUP-Texte für Absende-/Empfangsort ---
        popup_info_dispatch = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        popup_info_destination = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        # --- POPUP für die Linie (GeoJSON) ---
        line_popup_html = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        # 4) Marker für den Absendeort (Rot)
        folium.Marker(
            location=[disp_lat, disp_lon],
            popup=folium.Popup(popup_info_dispatch, max_width=300),
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(my_map)

        # 5) Marker für den Empfangsort (Blau)
        folium.Marker(
            location=[dest_lat, dest_lon],
            popup=folium.Popup(popup_info_destination, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(my_map)

        # 6) Statt folium.PolyLine nutzen wir nun GeoJSON, damit ein Klick auf die Linie ein Popup zeigt
        add_line_geojson(
            map_obj=my_map,
            lat1=disp_lat, lon1=disp_lon,
            lat2=dest_lat, lon2=dest_lon,
            popup_html=line_popup_html
        )

    return my_map


if __name__ == "__main__":
    # Hier ersetzt du df durch deinen DataFrame "rom_korr_map".
    # Beispiel:
    # rom_korr_map = pd.read_csv("rom_korr_map.csv")
    my_map = generate_map(rom_korr_map)

    # Dann kannst du die Karte speichern:
    my_map.save("romantik_map_geojson.html")
    pass


TypeError: Object of type function is not JSON serializable

In [1]:
#### Das hier ist die aktuelle Karte ####

import folium
import pandas as pd
from folium.features import GeoJson, GeoJsonPopup

def add_line_geojson(map_obj, lat1, lon1, lat2, lon2, popup_html):
    """
    Erzeugt ein GeoJSON-LineString-Feature, das klickbar ist
    und ein Popup anzeigt.
    Achtung: In GeoJSON wird [LON, LAT] genutzt!
    """
    # GeoJSON-Feature als Dictionary
    geojson_feature = {
        "type": "Feature",
        "properties": {
            "popup": popup_html  # Hier legen wir den Popup-Text ab
        },
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [lon1, lat1],
                [lon2, lat2]
            ]
        }
    }

    # GeoJsonPopup kümmert sich um die Darstellung der Property "popup"
    popup = GeoJsonPopup(
        fields=["popup"],  # Welche Properties sollen angezeigt werden?
        aliases=[""],       # Falls du einen Label vor "popup" möchtest, ansonsten leer lassen
        labels=False,       # Keine Feldnamenanzeige
        localize=False,     
        sticky=True,        # Popup bleibt beim Reinscrollen bestehen
        max_width=300
    )

    # Stil für die Linie
    def style_function(feature):
        return {
            "color": "green",
            "weight": 2,
            "opacity": 0.6
        }

    # Erzeuge ein Folium-GeoJson-Objekt
    geojson_obj = GeoJson(
        data=geojson_feature,
        style_function=style_function,
        name="Line",
        popup=popup,   # Popup "verknüpfen"
        tooltip="Klicken für mehr Info"  # Text beim Hover
    )

    geojson_obj.add_to(map_obj)


def generate_map(df):
    """
    Erzeugt eine interaktive Folium-Karte aus einem DataFrame df,
    der folgende Spalten enthält:
      - Date
      - Sender
      - Recipient
      - Place of Dispatch, Dispatch_Lat, Dispatch_Lon
      - Place of Destination, Destination_Lat, Destination_Lon
      - Distance_km (optional)
      - link (optional)

    Die Popups zeigen:
      - Überschrift: "Absender an Empfänger"
      - Absende- und Empfangsort
      - Datum
      - Distanz (falls vorhanden)
      - Link zur Originalseite

    Und zwar sowohl auf den Markern als auch direkt auf der Linie via GeoJSON.
    """

    # 1) Kartenmittelpunkt bestimmen
    valid_lat = df['Dispatch_Lat'].dropna().tolist() + df['Destination_Lat'].dropna().tolist()
    valid_lon = df['Dispatch_Lon'].dropna().tolist() + df['Destination_Lon'].dropna().tolist()

    if len(valid_lat) > 0 and len(valid_lon) > 0:
        avg_lat = sum(valid_lat) / len(valid_lat)
        avg_lon = sum(valid_lon) / len(valid_lon)
    else:
        # Fallback, wenn keine Koordinaten
        avg_lat, avg_lon = 50.0, 10.0

    # 2) Folium-Karte
    my_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    # 3) Schleife über DataFrame
    for idx, row in df.iterrows():
        disp_lat = row.get("Dispatch_Lat")
        disp_lon = row.get("Dispatch_Lon")
        dest_lat = row.get("Destination_Lat")
        dest_lon = row.get("Destination_Lon")

        # Falls Koordinaten fehlen, überspringen
        if pd.isna(disp_lat) or pd.isna(disp_lon) or pd.isna(dest_lat) or pd.isna(dest_lon):
            continue

        # Aus dem DF gelesene Felder
        date_str = row.get('Date', 'Unknown')
        sender = row.get('Sender', 'Unknown')
        recipient = row.get('Recipient', 'Unknown')
        place_disp = row.get('Place of Dispatch', 'Unknown')
        place_dest = row.get('Place of Destination', 'Unknown')
        link_str = row.get('link', '#')

        # Distanz (optional)
        distance = row.get("Distance_km")
        dist_info = ""
        if pd.notnull(distance):
            dist_info = f"Distanz: ~{distance:.1f} km<br/>"

        # Überschrift "Absender an Empfänger"
        heading = f"{sender} <b>an</b> {recipient}"

        # --- POPUP für Marker (Absende-/Empfangsort) ---
        popup_info_dispatch = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        popup_info_destination = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        # Popup für die Linie
        line_popup_html = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        # 4) Marker Absendeort (Rot)
        folium.Marker(
            location=[disp_lat, disp_lon],
            popup=folium.Popup(popup_info_dispatch, max_width=300),
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(my_map)

        # 5) Marker Empfangsort (Blau)
        folium.Marker(
            location=[dest_lat, dest_lon],
            popup=folium.Popup(popup_info_destination, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(my_map)

        # 6) Linie als GeoJSON -> Popup bei Klick auf die Linie
        add_line_geojson(
            map_obj=my_map,
            lat1=disp_lat, lon1=disp_lon,
            lat2=dest_lat, lon2=dest_lon,
            popup_html=line_popup_html
        )

    return my_map


if __name__ == "__main__":
    # Hier: Beliebiges Beispiel, ersetze es durch deinen DataFrame "rom_korr_map"
    rom_korr_full_website_coords = pd.read_csv("rom_korr_full_website_coords.csv")
    my_map = generate_map(rom_korr_full_website_coords)
    my_map.save("romantik_map_geojson.html")
    pass


In [1]:
###Tests

In [2]:
import folium
import pandas as pd
from folium.features import GeoJson, GeoJsonPopup
from folium.plugins import MarkerCluster

def add_line_geojson(map_obj, lat1, lon1, lat2, lon2, popup_html):
    """
    Erzeugt ein GeoJSON-LineString-Feature, das klickbar ist
    und ein Popup anzeigt.
    Achtung: In GeoJSON wird [LON, LAT] genutzt!
    """
    geojson_feature = {
        "type": "Feature",
        "properties": {
            "popup": popup_html
        },
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [lon1, lat1],
                [lon2, lat2]
            ]
        }
    }

    popup = GeoJsonPopup(
        fields=["popup"],  # Welche Properties als Popup angezeigt werden
        aliases=[""],      
        labels=False,
        localize=False,
        sticky=True,
        max_width=300
    )

    def style_function(feature):
        return {
            "color": "green",
            "weight": 2,
            "opacity": 0.6
        }

    geojson_obj = GeoJson(
        data=geojson_feature,
        style_function=style_function,
        name="Line",
        popup=popup,
        tooltip="Klicken für mehr Info"
    )

    geojson_obj.add_to(map_obj)


def generate_map(df):
    """
    Erzeugt eine interaktive Folium-Karte aus einem DataFrame df,
    der folgende Spalten enthält:
      - Date
      - Sender
      - Recipient
      - Place of Dispatch, Dispatch_Lat, Dispatch_Lon
      - Place of Destination, Destination_Lat, Destination_Lon
      - Distance_km (optional)
      - link (optional)

    Merkmale:
      - Marker-Cluster (Spiderfy) für alle Marker
      - Linien (GeoJSON), die bei Klick ein Popup öffnen
      - Popup-Texte bei Markern und Linien mit "Absender an Empfänger"
    """

    # 1) Kartenmittelpunkt bestimmen
    valid_lat = df['Dispatch_Lat'].dropna().tolist() + df['Destination_Lat'].dropna().tolist()
    valid_lon = df['Dispatch_Lon'].dropna().tolist() + df['Destination_Lon'].dropna().tolist()

    if len(valid_lat) > 0 and len(valid_lon) > 0:
        avg_lat = sum(valid_lat) / len(valid_lat)
        avg_lon = sum(valid_lon) / len(valid_lon)
    else:
        avg_lat, avg_lon = 50.0, 10.0  # Fallback

    # 2) Folium-Karte
    my_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    # 2a) MarkerCluster anlegen, mit Spiderfy-Option
    #     Hier kann man weitere Optionen einstellen: "disableClusteringAtZoom", etc.
    marker_cluster = MarkerCluster(
        options={
            "spiderfyOnEveryZoom": True,
            "showCoverageOnHover": True
        }
    )
    marker_cluster.add_to(my_map)

    # 3) Schleife über DataFrame
    for idx, row in df.iterrows():
        disp_lat = row.get("Dispatch_Lat")
        disp_lon = row.get("Dispatch_Lon")
        dest_lat = row.get("Destination_Lat")
        dest_lon = row.get("Destination_Lon")

        # Falls Koordinaten fehlen, überspringen
        if pd.isna(disp_lat) or pd.isna(disp_lon) or pd.isna(dest_lat) or pd.isna(dest_lon):
            continue

        # Felder aus dem DF
        date_str = row.get('Date', 'Unknown')
        sender = row.get('Sender', 'Unknown')
        recipient = row.get('Recipient', 'Unknown')
        place_disp = row.get('Place of Dispatch', 'Unknown')
        place_dest = row.get('Place of Destination', 'Unknown')
        link_str = row.get('link', '#')

        # Distanz (optional)
        distance = row.get("Distance_km")
        dist_info = ""
        if pd.notnull(distance):
            dist_info = f"Distanz: ~{distance:.1f} km<br/>"

        heading = f"{sender} <b>an</b> {recipient}"

        # Popup-Infos für die Marker
        popup_info_dispatch = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        popup_info_destination = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        # Popup für die Linie
        line_popup_html = f"""
        <b>{heading}</b><br/>
        <i>Absendeort:</i> {place_disp}<br/>
        <i>Empfangsort:</i> {place_dest}<br/>
        <i>Datum:</i> {date_str}<br/>
        {dist_info}
        <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
        """

        # 4) Roten Marker im Cluster für den Absendeort
        folium.Marker(
            location=[disp_lat, disp_lon],
            popup=folium.Popup(popup_info_dispatch, max_width=300),
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(marker_cluster)

        # 5) Blauen Marker im Cluster für den Empfangsort
        folium.Marker(
            location=[dest_lat, dest_lon],
            popup=folium.Popup(popup_info_destination, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(marker_cluster)

        # 6) Linie mit GeoJSON
        add_line_geojson(
            map_obj=my_map,
            lat1=disp_lat, lon1=disp_lon,
            lat2=dest_lat, lon2=dest_lon,
            popup_html=line_popup_html
        )

    return my_map


if __name__ == "__main__":
    # Beispiel:
    df = pd.read_csv("rom_korr_full_website_coords.csv")
    my_map = generate_map(df)
    my_map.save("romantik_map_spiderfy.html")
    pass


In [5]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, HeatMap
from folium.features import GeoJson, GeoJsonPopup

##########################################
# FUNKTION: DATAFRAME FILTERN
##########################################
def filter_df(
    df, 
    sender_filter=None, 
    recipient_filter=None, 
    year_min=None, 
    year_max=None
):
    """
    Filtert den DataFrame df nach:
      - Sender (falls sender_filter gesetzt und im 'Sender' enthalten),
      - Recipient (falls recipient_filter gesetzt und im 'Recipient' enthalten),
      - Year (gelesen aus Spalte 'Date', falls year_min / year_max angegeben).
    
    Rückgabe: gefilterter df
    """
    temp = df.copy()

    # 1) Sender-Filter (Substring-Suche im Feld 'Sender')
    if sender_filter:
        temp = temp[temp['Sender'].str.contains(sender_filter, case=False, na=False)]

    # 2) Recipient-Filter
    if recipient_filter:
        temp = temp[temp['Recipient'].str.contains(recipient_filter, case=False, na=False)]

    # 3) Jahres-Filter
    #  - Dafür brauchen wir eine Jahresspalte. Du kannst per Datumsparse oder per Regex das Jahr aus 'Date' ziehen.
    #  - Hier sehr simpel: Wir nehmen nur die letzten 4 Ziffern als Jahr.
    def extract_year(date_str):
        # Grob: Suche 4 zusammenhängende Ziffern
        import re
        match = re.search(r'(\d{4})', str(date_str))
        if match:
            return int(match.group(1))
        return None

    if year_min or year_max:
        if 'Year' not in temp.columns:
            temp['Year'] = temp['Date'].apply(extract_year)
        if year_min:
            temp = temp[temp['Year'] >= year_min]
        if year_max:
            temp = temp[temp['Year'] <= year_max]

    return temp


##########################################
# FUNKTION: GEOJSON-LINIE (Popups)
##########################################
def add_line_geojson(map_obj, lat1, lon1, lat2, lon2, popup_html):
    """
    Erzeugt ein GeoJSON-LineString-Feature, das klickbar ist
    und ein Popup anzeigt. [LON, LAT]-Reihenfolge beachtet.
    """
    geojson_feature = {
        "type": "Feature",
        "properties": {
            "popup": popup_html
        },
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [lon1, lat1],
                [lon2, lat2]
            ]
        }
    }

    popup = GeoJsonPopup(fields=["popup"], labels=False, max_width=300)

    def style_function(feature):
        return {
            "color": "green",
            "weight": 2,
            "opacity": 0.6
        }

    geojson_obj = GeoJson(
        data=geojson_feature,
        style_function=style_function,
        name="Line",
        popup=popup,
        tooltip="Klicken für mehr Info"
    )

    geojson_obj.add_to(map_obj)


##########################################
# FUNKTION: ERZEUGT HEATMAP-LAYER
##########################################
def add_heatmap_layer(df, map_obj, layer_name="Heatmap"):
    """
    Legt einen Heatmap-Layer über alle Dispatch- und Destination-Koordinaten.
    """
    heat_data = []

    for idx, row in df.iterrows():
        if pd.notnull(row['Dispatch_Lat']) and pd.notnull(row['Dispatch_Lon']):
            heat_data.append([row['Dispatch_Lat'], row['Dispatch_Lon']])
        if pd.notnull(row['Destination_Lat']) and pd.notnull(row['Destination_Lon']):
            heat_data.append([row['Destination_Lat'], row['Destination_Lon']])

    if len(heat_data) > 0:
        heat_layer = HeatMap(heat_data, name=layer_name, radius=10, blur=15)
        heat_layer.add_to(map_obj)

##########################################
# FUNKTION: SAMMELPOPUP FÜR VIELE EINTRÄGE
##########################################
def create_sammelpopup_html(group_df):
    """
    group_df = alle Briefe, die an derselben Koordinate liegen.
    Gibt einen HTML-String zurück, der alle Einträge in einer Liste darstellt.
    """
    # Evtl. nur 50 oder 100 Zeilen wirklich auflisten oder scollable <div>:
    items_html = []
    for idx, row in group_df.iterrows():
        heading = f"{row.get('Sender','?')} -> {row.get('Recipient','?')}"
        date_str = row.get('Date','?')
        link_str = row.get('link','#')
        items_html.append(f"<li><b>{heading}</b>, {date_str} [<a href='{link_str}' target='_blank'>Original</a>]</li>")
    items_joined = "\n".join(items_html)
    
    # In scrollbaren Div packen (falls es wirklich sehr lang ist):
    html = f"""
    <h4>Sammelmarker ({len(group_df)} Einträge)</h4>
    <div style="max-height:200px; overflow:auto;">
    <ul>
    {items_joined}
    </ul>
    </div>
    """
    return html

##########################################
# FUNKTION: HAUPTMAP ERZEUGEN
##########################################
def generate_map(df):
    """
    1) Heatmap-Layer für die gesamte Korrespondenz.
    2) Marker und Linien in separatem Layer (so kann man manuell umschalten).
    3) Bei >100 Einträgen an einem Ort => Sammelmarker (HTML-Liste).
       Sonst: spiderfy (MarkerCluster).
    """

    # -- 1) Kartenmittelpunkt
    valid_lat = df['Dispatch_Lat'].dropna().tolist() + df['Destination_Lat'].dropna().tolist()
    valid_lon = df['Dispatch_Lon'].dropna().tolist() + df['Destination_Lon'].dropna().tolist()
    if len(valid_lat) > 0 and len(valid_lon) > 0:
        avg_lat = sum(valid_lat) / len(valid_lat)
        avg_lon = sum(valid_lon) / len(valid_lon)
    else:
        avg_lat, avg_lon = 50.0, 10.0

    # -- 2) Grundkarte
    my_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    # -- 2a) Heatmap-Layer anlegen
    add_heatmap_layer(df, my_map, layer_name="Briefe Heatmap")

    # -- 2b) FeatureGroup für Marker & Linien (damit wir in LayerControl togglen können)
    marker_line_group = folium.FeatureGroup(name="Marker & Linien")
    marker_line_group.add_to(my_map)

    # -- 2c) MarkerCluster für spiderfy, ABER wir zeigen nur Marker <= 100 Einträge pro Ort.
    cluster = MarkerCluster(
        options={
            "spiderfyOnEveryZoom": True,
            "showCoverageOnHover": True
        }
    )
    cluster.add_to(marker_line_group)

    # -- 3) Wir gruppieren nach Koordinaten, um zu sehen, wie viele Einträge an einer Stelle liegen.
    #       Machen wir für Dispatch UND Destination separat? => In diesem Beispiel nur Dispatch.
    #       Dann analog für Destination. Oder du packst "Absendeort" und "Empfangsort"
    #       in eine unify-Tabelle. Hier: Nur Dispatch als Beispiel.
    
    # 3a) Erstellen wir eine Hilfsspalte, um Dispatch-Koords zu gruppieren:
    df['_dispatch_coord'] = df['Dispatch_Lat'].astype(str) + "_" + df['Dispatch_Lon'].astype(str)
    df['_destination_coord'] = df['Destination_Lat'].astype(str) + "_" + df['Destination_Lon'].astype(str)

    # So können wir für Dispatch das groupby machen und für Destination das groupby machen:
    dispatch_groups = df.groupby('_dispatch_coord')
    dest_groups = df.groupby('_destination_coord')

    # Wir merken uns: -> dictionary, key= Koord, value= DataFrame der Einträge
    dispatch_dict = {g: dispatch_groups.get_group(g) for g in dispatch_groups.groups}
    dest_dict = {g: dest_groups.get_group(g) for g in dest_groups.groups}

    # 3b) Helper: Hol Koordinaten aus so einer Gruppe
    def get_lat_lon_from_groupkey(gkey, is_dispatch=True):
        parts = gkey.split("_")
        if len(parts)==2:
            lat_s, lon_s = parts
            try:
                lat_f = float(lat_s)
                lon_f = float(lon_s)
                return lat_f, lon_f
            except:
                pass
        return None, None

    # -- 4) Für LINIEN brauchen wir pro Zeile -> wir iterieren im Original-DF
    #       Und fügen in marker_line_group per add_line_geojson(...) die Verbindung ein.
    for idx, row in df.iterrows():
        lat1 = row['Dispatch_Lat']
        lon1 = row['Dispatch_Lon']
        lat2 = row['Destination_Lat']
        lon2 = row['Destination_Lon']

        if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
            sender = row.get("Sender", "?")
            recipient = row.get("Recipient", "?")
            place_disp = row.get("Place of Dispatch","?")
            place_dest = row.get("Place of Destination","?")
            date_str = row.get("Date","?")
            link_str = row.get("link","#")
            distance = row.get("Distance_km")
            dist_info = ""
            if pd.notnull(distance):
                dist_info = f"Distanz: ~{distance:.1f} km<br/>"
            heading = f"{sender} <b>an</b> {recipient}"

            line_popup_html = f"""
            <b>{heading}</b><br/>
            <i>Absendeort:</i> {place_disp}<br/>
            <i>Empfangsort:</i> {place_dest}<br/>
            <i>Datum:</i> {date_str}<br/>
            {dist_info}
            <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
            """
            add_line_geojson(marker_line_group, lat1, lon1, lat2, lon2, line_popup_html)


    # -- 5) Marker anlegen: Dispatch + Destination
    #    a) Dispatch: Wir schauen in dispatch_dict, ob len > 100 => Sammelmarker
    #    b) Sonst => je Eintrag Marker
    #    ( dasselbe für Destination )

    # 5a) Dispatch
    for coord_key, group_df in dispatch_dict.items():
        # Koords:
        lat, lon = get_lat_lon_from_groupkey(coord_key, is_dispatch=True)
        if pd.isna(lat) or pd.isna(lon):
            continue

        count_entries = len(group_df)
        if count_entries > 100:
            # Ein Sammelmarker
            sammel_popup = create_sammelpopup_html(group_df)
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(sammel_popup, max_width=300),
                icon=folium.Icon(color="cadetblue", icon="info-sign", prefix="fa")
            ).add_to(marker_line_group)  # Keine Spiderfy => direkter Marker
        else:
            # Weniger gleich 100 => alle einzeln im Cluster
            # -> Wir iterieren über group_df
            for idx2, row2 in group_df.iterrows():
                sender = row2.get("Sender", "?")
                recipient = row2.get("Recipient", "?")
                date_str = row2.get("Date","?")
                link_str = row2.get("link","#")
                place_disp = row2.get("Place of Dispatch","?")
                heading = f"{sender} <b>an</b> {recipient}"
                popup_text = f"""
                <b>{heading}</b><br/>
                <i>Absendeort:</i> {place_disp}<br/>
                <i>Datum:</i> {date_str}<br/>
                <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
                """
                folium.Marker(
                    location=[lat, lon],
                    popup=folium.Popup(popup_text, max_width=300),
                    icon=folium.Icon(color='red', icon='info-sign')
                ).add_to(cluster)

    # 5b) Destination
    for coord_key, group_df in dest_dict.items():
        lat, lon = get_lat_lon_from_groupkey(coord_key, is_dispatch=False)
        if pd.isna(lat) or pd.isna(lon):
            continue

        count_entries = len(group_df)
        if count_entries > 100:
            # Ein Sammelmarker
            sammel_popup = create_sammelpopup_html(group_df)
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(sammel_popup, max_width=300),
                icon=folium.Icon(color="darkblue", icon="info-sign", prefix="fa")
            ).add_to(marker_line_group)
        else:
            # <= 100 => spiderfy (Cluster)
            for idx2, row2 in group_df.iterrows():
                sender = row2.get("Sender", "?")
                recipient = row2.get("Recipient", "?")
                date_str = row2.get("Date","?")
                link_str = row2.get("link","#")
                place_dest = row2.get("Place of Destination","?")
                heading = f"{sender} <b>an</b> {recipient}"
                popup_text = f"""
                <b>{heading}</b><br/>
                <i>Empfangsort:</i> {place_dest}<br/>
                <i>Datum:</i> {date_str}<br/>
                <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
                """
                folium.Marker(
                    location=[lat, lon],
                    popup=folium.Popup(popup_text, max_width=300),
                    icon=folium.Icon(color='blue', icon='info-sign')
                ).add_to(cluster)

    # -- 6) LayerControl hinzufügen, damit man Heatmap vs. Marker/Linien umschalten kann
    folium.LayerControl().add_to(my_map)

    return my_map


##########################################
# BEISPIELHAFTES HAUPTPROGRAMM
##########################################
if __name__ == "__main__":
    # 1) Daten laden
    df = pd.read_csv("rom_korr_full_website_coords.csv")

    # 2) Vor-Filter angeben (Optional):
    #    z.B. nur Briefe zwischen 1790 und 1800, Sender "Schlegel", Empfänger "Heyne"
    # df = filter_df(df, sender_filter="Schlegel", recipient_filter="Heyne", year_min=1790, year_max=1800)

    # 3) Karte erzeugen
    my_map = generate_map(df)

    # 4) Karte speichern
    my_map.save("romantik_map_spiderfy_zoom.html")
    print("Karte gespeichert: romantik_map_spiderfy_zoom.html")


Karte gespeichert: romantik_map_spiderfy_zoom.html


In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, HeatMap
from folium.features import GeoJson, GeoJsonPopup

##########################################
# FUNKTION: DATAFRAME FILTERN
##########################################
def filter_df(
    df, 
    sender_filter=None, 
    recipient_filter=None, 
    year_min=None, 
    year_max=None
):
    """
    Filtert den DataFrame df nach:
      - Sender (falls sender_filter gesetzt und im 'Sender' enthalten),
      - Recipient (falls recipient_filter gesetzt und im 'Recipient' enthalten),
      - Year (gelesen aus Spalte 'Date', falls year_min / year_max angegeben).
    
    Rückgabe: gefilterter df
    """
    temp = df.copy()

    # 1) Sender-Filter (Substring-Suche im Feld 'Sender')
    if sender_filter:
        temp = temp[temp['Sender'].str.contains(sender_filter, case=False, na=False)]

    # 2) Recipient-Filter
    if recipient_filter:
        temp = temp[temp['Recipient'].str.contains(recipient_filter, case=False, na=False)]

    # 3) Jahres-Filter
    #  - Dafür brauchen wir eine Jahresspalte. Du kannst per Datumsparse oder per Regex das Jahr aus 'Date' ziehen.
    #  - Hier sehr simpel: Wir nehmen nur die letzten 4 Ziffern als Jahr.
    def extract_year(date_str):
        # Grob: Suche 4 zusammenhängende Ziffern
        import re
        match = re.search(r'(\d{4})', str(date_str))
        if match:
            return int(match.group(1))
        return None

    if year_min or year_max:
        if 'Year' not in temp.columns:
            temp['Year'] = temp['Date'].apply(extract_year)
        if year_min:
            temp = temp[temp['Year'] >= year_min]
        if year_max:
            temp = temp[temp['Year'] <= year_max]

    return temp


##########################################
# FUNKTION: GEOJSON-LINIE (Popups)
##########################################
def add_line_geojson(map_obj, lat1, lon1, lat2, lon2, popup_html):
    """
    Erzeugt ein GeoJSON-LineString-Feature, das klickbar ist
    und ein Popup anzeigt. [LON, LAT]-Reihenfolge beachtet.
    """
    geojson_feature = {
        "type": "Feature",
        "properties": {
            "popup": popup_html
        },
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [lon1, lat1],
                [lon2, lat2]
            ]
        }
    }

    popup = GeoJsonPopup(fields=["popup"], labels=False, max_width=300)

    def style_function(feature):
        return {
            "color": "green",
            "weight": 2,
            "opacity": 0.6
        }

    geojson_obj = GeoJson(
        data=geojson_feature,
        style_function=style_function,
        name="Line",
        popup=popup,
        tooltip="Klicken für mehr Info"
    )

    geojson_obj.add_to(map_obj)


##########################################
# FUNKTION: ERZEUGT HEATMAP-LAYER
##########################################
def add_heatmap_layer(df, map_obj, layer_name="Heatmap"):
    """
    Legt einen Heatmap-Layer über alle Dispatch- und Destination-Koordinaten.
    """
    heat_data = []

    for idx, row in df.iterrows():
        if pd.notnull(row['Dispatch_Lat']) and pd.notnull(row['Dispatch_Lon']):
            heat_data.append([row['Dispatch_Lat'], row['Dispatch_Lon']])
        if pd.notnull(row['Destination_Lat']) and pd.notnull(row['Destination_Lon']):
            heat_data.append([row['Destination_Lat'], row['Destination_Lon']])

    if len(heat_data) > 0:
        heat_layer = HeatMap(heat_data, name=layer_name, radius=20, blur=5, min_opacity=0.4)
        heat_layer.add_to(map_obj)

##########################################
# FUNKTION: SAMMELPOPUP FÜR VIELE EINTRÄGE
##########################################
def create_sammelpopup_html(group_df):
    """
    group_df = alle Briefe, die an derselben Koordinate liegen.
    Gibt einen HTML-String zurück, der alle Einträge in einer Liste darstellt.
    """
    # Evtl. nur 50 oder 100 Zeilen wirklich auflisten oder scollable <div>:
    items_html = []
    for idx, row in group_df.iterrows():
        heading = f"{row.get('Sender','?')} -> {row.get('Recipient','?')}"
        date_str = row.get('Date','?')
        link_str = row.get('link','#')
        items_html.append(f"<li><b>{heading}</b>, {date_str} [<a href='{link_str}' target='_blank'>Original</a>]</li>")
    items_joined = "\n".join(items_html)
    
    # In scrollbaren Div packen (falls es wirklich sehr lang ist):
    html = f"""
    <h4>Sammelmarker ({len(group_df)} Einträge)</h4>
    <div style="max-height:200px; overflow:auto;">
    <ul>
    {items_joined}
    </ul>
    </div>
    """
    return html

##########################################
# FUNKTION: HAUPTMAP ERZEUGEN
##########################################
def generate_map(df):
    """
    1) Heatmap-Layer für die gesamte Korrespondenz.
    2) Marker und Linien in separatem Layer (so kann man manuell umschalten).
    3) Bei >100 Einträgen an einem Ort => Sammelmarker (HTML-Liste).
       Sonst: spiderfy (MarkerCluster).
    """

    # -- 1) Kartenmittelpunkt
    valid_lat = df['Dispatch_Lat'].dropna().tolist() + df['Destination_Lat'].dropna().tolist()
    valid_lon = df['Dispatch_Lon'].dropna().tolist() + df['Destination_Lon'].dropna().tolist()
    if len(valid_lat) > 0 and len(valid_lon) > 0:
        avg_lat = sum(valid_lat) / len(valid_lat)
        avg_lon = sum(valid_lon) / len(valid_lon)
    else:
        avg_lat, avg_lon = 50.0, 10.0

    # -- 2) Grundkarte
    my_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    # -- 2a) Heatmap-Layer anlegen
    add_heatmap_layer(df, my_map, layer_name="Briefe Heatmap")

    # -- 2b) FeatureGroup für Marker & Linien (damit wir in LayerControl togglen können)
    marker_line_group = folium.FeatureGroup(name="Marker & Linien")
    marker_line_group.add_to(my_map)

    # -- 2c) MarkerCluster für spiderfy, ABER wir zeigen nur Marker <= 100 Einträge pro Ort.
    cluster = MarkerCluster(
        options={
            "spiderfyOnEveryZoom": True,
            "showCoverageOnHover": True
        }
    )
    cluster.add_to(marker_line_group)

    # -- 3) Wir gruppieren nach Koordinaten, um zu sehen, wie viele Einträge an einer Stelle liegen.
    #       Machen wir für Dispatch UND Destination separat? => In diesem Beispiel nur Dispatch.
    #       Dann analog für Destination. Oder du packst "Absendeort" und "Empfangsort"
    #       in eine unify-Tabelle. Hier: Nur Dispatch als Beispiel.
    
    # 3a) Erstellen wir eine Hilfsspalte, um Dispatch-Koords zu gruppieren:
    df['_dispatch_coord'] = df['Dispatch_Lat'].astype(str) + "_" + df['Dispatch_Lon'].astype(str)
    df['_destination_coord'] = df['Destination_Lat'].astype(str) + "_" + df['Destination_Lon'].astype(str)

    # So können wir für Dispatch das groupby machen und für Destination das groupby machen:
    dispatch_groups = df.groupby('_dispatch_coord')
    dest_groups = df.groupby('_destination_coord')

    # Wir merken uns: -> dictionary, key= Koord, value= DataFrame der Einträge
    dispatch_dict = {g: dispatch_groups.get_group(g) for g in dispatch_groups.groups}
    dest_dict = {g: dest_groups.get_group(g) for g in dest_groups.groups}

    # 3b) Helper: Hol Koordinaten aus so einer Gruppe
    def get_lat_lon_from_groupkey(gkey, is_dispatch=True):
        parts = gkey.split("_")
        if len(parts)==2:
            lat_s, lon_s = parts
            try:
                lat_f = float(lat_s)
                lon_f = float(lon_s)
                return lat_f, lon_f
            except:
                pass
        return None, None

    # -- 4) Für LINIEN brauchen wir pro Zeile -> wir iterieren im Original-DF
    #       Und fügen in marker_line_group per add_line_geojson(...) die Verbindung ein.
    for idx, row in df.iterrows():
        lat1 = row['Dispatch_Lat']
        lon1 = row['Dispatch_Lon']
        lat2 = row['Destination_Lat']
        lon2 = row['Destination_Lon']

        if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
            sender = row.get("Sender", "?")
            recipient = row.get("Recipient", "?")
            place_disp = row.get("Place of Dispatch","?")
            place_dest = row.get("Place of Destination","?")
            date_str = row.get("Date","?")
            link_str = row.get("link","#")
            distance = row.get("Distance_km")
            dist_info = ""
            if pd.notnull(distance):
                dist_info = f"Distanz: ~{distance:.1f} km<br/>"
            heading = f"{sender} <b>an</b> {recipient}"

            line_popup_html = f"""
            <b>{heading}</b><br/>
            <i>Absendeort:</i> {place_disp}<br/>
            <i>Empfangsort:</i> {place_dest}<br/>
            <i>Datum:</i> {date_str}<br/>
            {dist_info}
            <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
            """
            add_line_geojson(marker_line_group, lat1, lon1, lat2, lon2, line_popup_html)


    # -- 5) Marker anlegen: Dispatch + Destination
    #    a) Dispatch: Wir schauen in dispatch_dict, ob len > 100 => Sammelmarker
    #    b) Sonst => je Eintrag Marker
    #    ( dasselbe für Destination )

    # 5a) Dispatch
    for coord_key, group_df in dispatch_dict.items():
        # Koords:
        lat, lon = get_lat_lon_from_groupkey(coord_key, is_dispatch=True)
        if pd.isna(lat) or pd.isna(lon):
            continue

        count_entries = len(group_df)
        if count_entries > 100:
            # Ein Sammelmarker
            sammel_popup = create_sammelpopup_html(group_df)
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(sammel_popup, max_width=300),
                icon=folium.Icon(color="cadetblue", icon="info-sign", prefix="fa")
            ).add_to(marker_line_group)  # Keine Spiderfy => direkter Marker
        else:
            # Weniger gleich 100 => alle einzeln im Cluster
            # -> Wir iterieren über group_df
            for idx2, row2 in group_df.iterrows():
                sender = row2.get("Sender", "?")
                recipient = row2.get("Recipient", "?")
                date_str = row2.get("Date","?")
                link_str = row2.get("link","#")
                place_disp = row2.get("Place of Dispatch","?")
                heading = f"{sender} <b>an</b> {recipient}"
                popup_text = f"""
                <b>{heading}</b><br/>
                <i>Absendeort:</i> {place_disp}<br/>
                <i>Datum:</i> {date_str}<br/>
                <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
                """
                folium.Marker(
                    location=[lat, lon],
                    popup=folium.Popup(popup_text, max_width=300),
                    icon=folium.Icon(color='red', icon='info-sign')
                ).add_to(cluster)

    # 5b) Destination
    for coord_key, group_df in dest_dict.items():
        lat, lon = get_lat_lon_from_groupkey(coord_key, is_dispatch=False)
        if pd.isna(lat) or pd.isna(lon):
            continue

        count_entries = len(group_df)
        if count_entries > 100:
            # Ein Sammelmarker
            sammel_popup = create_sammelpopup_html(group_df)
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(sammel_popup, max_width=300),
                icon=folium.Icon(color="darkblue", icon="info-sign", prefix="fa")
            ).add_to(marker_line_group)
        else:
            # <= 100 => spiderfy (Cluster)
            for idx2, row2 in group_df.iterrows():
                sender = row2.get("Sender", "?")
                recipient = row2.get("Recipient", "?")
                date_str = row2.get("Date","?")
                link_str = row2.get("link","#")
                place_dest = row2.get("Place of Destination","?")
                heading = f"{sender} <b>an</b> {recipient}"
                popup_text = f"""
                <b>{heading}</b><br/>
                <i>Empfangsort:</i> {place_dest}<br/>
                <i>Datum:</i> {date_str}<br/>
                <i>Link:</i> <a href="{link_str}" target="_blank">Original</a>
                """
                folium.Marker(
                    location=[lat, lon],
                    popup=folium.Popup(popup_text, max_width=300),
                    icon=folium.Icon(color='blue', icon='info-sign')
                ).add_to(cluster)

    # -- 6) LayerControl hinzufügen, damit man Heatmap vs. Marker/Linien umschalten kann
    folium.LayerControl().add_to(my_map)

    return my_map


##########################################
# BEISPIELHAFTES HAUPTPROGRAMM
##########################################
if __name__ == "__main__":
    # 1) Daten laden
    df = pd.read_csv("rom_korr_full_website_coords.csv")

    # 2) Vor-Filter angeben (Optional):
    #    z.B. nur Briefe zwischen 1790 und 1800, Sender "Schlegel", Empfänger "Heyne"
    # df = filter_df(df, sender_filter="Schlegel", recipient_filter="Heyne", year_min=1790, year_max=1800)

    # 3) Karte erzeugen
    my_map = generate_map(df)

    # 4) Karte speichern
    my_map.save("romantik_map_heat_spiderfy.html")
    print("Karte gespeichert")


Karte gespeichert: romantik_map_spiderfy_zoom.html
